In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install autogluon

import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from autogluon.timeseries.utils.forecast import get_forecast_horizon_index_ts_dataframe


In [126]:
# 데이터 가져오기

train_file_path = '/content/drive/MyDrive/open-2/train.csv'
test_file_path = '/content/drive/MyDrive/open-2/test.csv'
sample_file_path = '/content/drive/MyDrive/open-2/sample_submission.csv'


df = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)
sample_data = pd.read_csv(sample_file_path)

print(df)

                    ID   timestamp item corporation location  supply(kg)  \
0      TG_A_J_20190101  2019-01-01   TG           A        J         0.0   
1      TG_A_J_20190102  2019-01-02   TG           A        J         0.0   
2      TG_A_J_20190103  2019-01-03   TG           A        J     60601.0   
3      TG_A_J_20190104  2019-01-04   TG           A        J     25000.0   
4      TG_A_J_20190105  2019-01-05   TG           A        J     32352.0   
...                ...         ...  ...         ...      ...         ...   
59392  RD_F_J_20230227  2023-02-27   RD           F        J    452440.0   
59393  RD_F_J_20230228  2023-02-28   RD           F        J    421980.0   
59394  RD_F_J_20230301  2023-03-01   RD           F        J    382980.0   
59395  RD_F_J_20230302  2023-03-02   RD           F        J    477220.0   
59396  RD_F_J_20230303  2023-03-03   RD           F        J    427520.0   

       price(원/kg)  
0              0.0  
1              0.0  
2           1728.0  
3  

In [128]:
# 요일 추가
df['timestamp'] = pd.to_datetime(df['timestamp'])


# 요일 정보 추출 (월요일=0, 일요일=6)
df['dayofweek'] = df['timestamp'].dt.dayofweek

# 요일 정보를 원-핫 인코딩으로 변환
df = pd.get_dummies(df, columns=['dayofweek'])

df.rename(columns={
    'dayofweek_0': 'monday',
    'dayofweek_1': 'tuesday',
    'dayofweek_2': 'wednesday',
    'dayofweek_3': 'thursday',
    'dayofweek_4': 'friday',
    'dayofweek_5': 'saturday',
    'dayofweek_6': 'sunday'
}, inplace=True)


In [129]:
df['price_lag_1'] = df['price(원/kg)'].shift(1)
df['price_lag_7'] = df['price(원/kg)'].shift(7)
df['price_lag_30'] = df['price(원/kg)'].shift(30)


In [131]:
# item원핫인코딩
df = pd.get_dummies(df, columns=['item'], prefix='', prefix_sep='')

df.head()

,ID,timestamp,corporation,location,supply(kg),price(원/kg),monday,tuesday,wednesday,thursday,...,price_lag_1,price_lag_7,price_lag_30,7_day_MA,7_day_STD,BC,CB,CR,RD,TG
0,TG_A_J_20190101,2019-01-01,A,J,0.0,0.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
1,TG_A_J_20190102,2019-01-02,A,J,0.0,0.0,0,0,1,0,...,0.0,NaN,NaN,NaN,NaN,0,0,0,0,1
2,TG_A_J_20190103,2019-01-03,A,J,60601.0,1728.0,0,0,0,1,...,0.0,NaN,NaN,NaN,NaN,0,0,0,0,1
3,TG_A_J_20190104,2019-01-04,A,J,25000.0,1408.0,0,0,0,0,...,1728.0,NaN,NaN,NaN,NaN,0,0,0,0,1
4,TG_A_J_20190105,2019-01-05,A,J,32352.0,1250.0,0,0,0,0,...,1408.0,NaN,NaN,NaN,NaN,0,0,0,0,1


In [132]:
df['item_id'] = df.ID.str[0:6]

train_data = TimeSeriesDataFrame(df.drop(columns=['ID'])) # 멀티 인덱스
train_data = pd.get_dummies(train_data, columns=['corporation', 'location'])

print(train_data)

                    supply(kg)  price(원/kg)  monday  tuesday  wednesday  \
item_id timestamp                                                         
TG_A_J  2019-01-01         0.0          0.0       0        1          0   
        2019-01-02         0.0          0.0       0        0          1   
        2019-01-03     60601.0       1728.0       0        0          0   
        2019-01-04     25000.0       1408.0       0        0          0   
        2019-01-05     32352.0       1250.0       0        0          0   
...                        ...          ...     ...      ...        ...   
RD_F_J  2023-02-27    452440.0        468.0       1        0          0   
        2023-02-28    421980.0        531.0       0        1          0   
        2023-03-01    382980.0        574.0       0        0          1   
        2023-03-02    477220.0        523.0       0        0          0   
        2023-03-03    427520.0        529.0       0        0          0   

                    thur

In [133]:
train_data['price_lag_1'].fillna(0, inplace=True)
train_data['price_lag_7'].fillna(0, inplace=True)
train_data['price_lag_30'].fillna(0, inplace=True)


print(train_data)

                    supply(kg)  price(원/kg)  monday  tuesday  wednesday  \
item_id timestamp                                                         
TG_A_J  2019-01-01         0.0          0.0       0        1          0   
        2019-01-02         0.0          0.0       0        0          1   
        2019-01-03     60601.0       1728.0       0        0          0   
        2019-01-04     25000.0       1408.0       0        0          0   
        2019-01-05     32352.0       1250.0       0        0          0   
...                        ...          ...     ...      ...        ...   
RD_F_J  2023-02-27    452440.0        468.0       1        0          0   
        2023-02-28    421980.0        531.0       0        1          0   
        2023-03-01    382980.0        574.0       0        0          1   
        2023-03-02    477220.0        523.0       0        0          0   
        2023-03-03    427520.0        529.0       0        0          0   

                    thur

In [134]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)

predictor.fit( train_data,
              random_seed=42,
              enable_ensemble = True,
)


================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231116_160817/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)', 

In [135]:
ld_board = predictor.leaderboard(train_data, silent=True)
ld_board

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


In [137]:
# Retrain model on all of the data (training + validation).
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	4.56    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 4.63 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [138]:
# Return quantile and mean forecasts for the given dataset, starting from the end of each time series.

pred = predictor.predict(train_data, random_seed=42,)
pred

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [140]:
sample_data['answer'] = pred.reset_index()['mean']
sample_data.loc[sample_data['answer'] < 0.0, 'answer'] = 0.0
sample_data


In [142]:

# 'ID' 열에서 날짜 정보 추출
sample_data['date'] = pd.to_datetime(sample_data['ID'].str[-8:])

# 일요일 여부 확인 (dayofweek에서 6은 일요일)
sample_data['is_sunday'] = sample_data['date'].dt.dayofweek == 6

# 일요일에 해당하는 행의 'answer' 값을 0.0으로 설정
sample_data.loc[sample_data['is_sunday'], 'answer'] = 0.0

# 'date'와 'is_sunday' 열 삭제 (더 이상 필요 없음)
sample_data.drop(columns=['date', 'is_sunday'], inplace=True)

# 결과 확인
print(sample_data.head())


                ID       answer
0  TG_A_J_20230304  3272.434810
1  TG_A_J_20230305     0.000000
2  TG_A_J_20230306  3107.874419
3  TG_A_J_20230307  3402.964269
4  TG_A_J_20230308  3358.904634


In [143]:

save_path = '/content/drive/My Drive/sundayautosubmission1117(7).csv'
sample_data.to_csv(save_path, index=False)

In [ ]:
print(predictor.evaluate(train_data))
print(predictor.fit_summary())
print(predictor.get_model_best())
print(predictor.info())

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL
